In [2]:
import pandas as pd
import json
from sklearn.feature_extraction.text import CountVectorizer


In [3]:
json_path = 'entity_dict_negated.json'

json_list = []

with open(json_path, 'r') as f:
    # read lines
    lines = f.readlines()
    # each line is a json object
    for line in lines:
        json_list.append(json.loads(line))
# to df
json_df = pd.DataFrame(json_list)

#len_list = []
#for i, row in json_df.iterrows():
#    sample_ent_list = [i for i in list(row['entities']) if i != '<br>']
#    len_list.append(len(sample_ent_list))


In [3]:
len(json_df)

2147

In [7]:
print(len(set(json_df['row_id'])))
print(len(json_df['row_id']))


16071
16077


In [ ]:
def return_entity_list(json_obj):
    entity_list = []
    row_id = json_obj['row_id']
    entities = json_obj['entities']
    # entit_list to dict
    entity_dict = {}
    for ent in entities:
        if ent[1] in entity_dict:
            entity_dict[ent[1]].append(ent[0])
        else:
            entity_dict[ent[1]] = [ent[0]]

    for key, value in entity_dict.items():   
        if key in {'CLINICAL_DEPT','OCCURRENCE','EVIDENTIAL','O'}:
            pass
        else:
            for entity in value:
                #entity_list.append(key+'_'+entity)
                # lowercase entity
                entity = entity.lower().strip()
                # delete ',' and ':' at the end of entity
                if entity[-1] in {',',':'}:
                    entity = entity.replace(',','').replace(':','')
                if entity[:2] == 'a ':
                    entity = entity[2:]
                entity_list.append('<type: '+key+'>'+entity)
                
    return entity_list, row_id

In [ ]:
doc_ent_list = []
id_list  = []

for json_obj in json_list:
    entity_list, id = return_entity_list(json_obj)
    entity_set = list(set(entity_list))
    doc_ent_list.append(entity_set)
    id_list.append(id)
    

In [ ]:
# Initialize CountVectorizer
vectorizer = CountVectorizer(analyzer=lambda x: x)

X = vectorizer.fit_transform(doc_ent_list[:10000])

feature_names = vectorizer.get_feature_names_out()

# X to df
df = pd.DataFrame(X.toarray(), columns=feature_names, index=id_list[:10000])

In [ ]:
len(df.columns)

411006

In [ ]:
# delete columns whose sum is less than threshold
threshold = 100
df = df.loc[:, df.sum() > threshold]

In [ ]:
len(df.columns)

2909

In [ ]:
# delete first 8 columns
df = df.iloc[:,8:]

In [ ]:
len(df)

10000

In [ ]:
# get unique vectors (rows) in the df
df_unique = df.drop_duplicates()

print(len(df_unique))

7304


In [ ]:
# to csv
df.to_csv('entity_matrix.csv')

In [ ]:
# average of sum of row vectors
df.sum(axis=1).mean()

99.6303